In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# from longchat.train.monkey_patch.llama_condense_monkey_patch import replace_llama_with_condense

# replace_llama_with_condense(ratio=1.5)

from longchat.train.monkey_patch.llama_flash_attn_monkey_patch import (
    replace_llama_attn_with_flash_attn,
)

replace_llama_attn_with_flash_attn()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/tianbai_cui_siriusxm_com/disk2/miniconda3/envs/torch/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/tianbai_cui_siriusxm_com/disk2/miniconda3/envs/torch/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /home/tianbai_cui_siriusxm_com/disk2/miniconda3/envs/torch did not contain libcudart.so as expected! Searching further paths...
  warn(msg)


[2023-07-31 16:33:19,422] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
base_model = AutoModelForCausalLM.from_pretrained(
    "openlm-research/open_llama_13b",
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [1]:
from pathlib import Path
peft_model_path = Path("~/disk2/workspace/llm_finetune/wizard-vicuna-dolly-samsum/checkpoint-800").expanduser()

In [4]:
peft_model_path

PosixPath('/home/tianbai_cui_siriusxm_com/disk2/workspace/llm_finetune/wizard-vicuna-dolly-samsum/checkpoint-800')

In [5]:
model = PeftModel.from_pretrained(base_model, peft_model_path, device_map='auto')

In [6]:
model = model.base_model.model

In [10]:
model.config

LlamaConfig {
  "_name_or_path": "tcui/open-vicuna",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 5120,
  "initializer_range": 0.02,
  "intermediate_size": 13824,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 40,
  "num_hidden_layers": 40,
  "num_key_value_heads": 40,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vocab_size": 32000
}

In [11]:
tokenizer = AutoTokenizer.from_pretrained("tcui/open-vicuna", use_fast=False, legacy=False)

In [12]:
peft_model_path.parent

PosixPath('/home/tianbai_cui_siriusxm_com/disk2/workspace/llm_finetune/wizard-vicuna-dolly-samsum')

In [13]:
model.save_pretrained(peft_model_path.parent / "HF_model")
tokenizer.save_pretrained(peft_model_path.parent / "HF_model")

('/home/tianbai_cui_siriusxm_com/disk2/workspace/llm_finetune/wizard-vicuna-dolly-samsum/HF_model/tokenizer_config.json',
 '/home/tianbai_cui_siriusxm_com/disk2/workspace/llm_finetune/wizard-vicuna-dolly-samsum/HF_model/special_tokens_map.json',
 '/home/tianbai_cui_siriusxm_com/disk2/workspace/llm_finetune/wizard-vicuna-dolly-samsum/HF_model/tokenizer.model',
 '/home/tianbai_cui_siriusxm_com/disk2/workspace/llm_finetune/wizard-vicuna-dolly-samsum/HF_model/added_tokens.json')

In [11]:
model.max_position_embeddings

3072

In [13]:
model.config.max_position_embeddings = int(2048*1.5)

In [14]:
model.config

LlamaConfig {
  "_name_or_path": "openlm-research/open_llama_13b",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 5120,
  "initializer_range": 0.02,
  "intermediate_size": 13824,
  "max_position_embeddings": 3072,
  "model_type": "llama",
  "num_attention_heads": 40,
  "num_hidden_layers": 40,
  "num_key_value_heads": 40,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vocab_size": 32000
}

In [21]:
model.base_model.model.save_pretrained(peft_model_path / "HF_model")

In [8]:
model.base_model.model.model.layers[0].self_attn.rotary_emb

CondenseRotaryEmbedding()

In [12]:
model.push_to_hub("OLlama-sft-65k", use_temp_dir=False)
tokenizer.push_to_hub("OLlama-sft-65k", use_temp_dir=False)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 model.push_to_hub("OLlama-sft-65k", use_temp_dir=False)                                      │
│   2 tokenizer.push_to_hub("OLlama-sft-65k", use_temp_dir=False)                                  │
│   3                                                                                              │
│                                                                                                  │
│ /home/tianbai_cui_siriusxm_com/disk2/miniconda3/envs/torch/lib/python3.10/site-packages/transfor │
│ mers/utils/hub.py:803 in push_to_hub                                                             │
│                                                                                                  │
│    800 │   │   else:                                                                             │
│    801 │   │   │   working_dir = repo_id.split("/")[-1]                                          │
│    802 │   │                                                                                     │
│ ❱  803 │   │   repo_id = self._create_repo(                                                      │
│    804 │   │   │   repo_id, private=private, use_auth_token=use_auth_token, repo_url=repo_url,   │
│    805 │   │   )                                                                                 │
│    806                                                                                           │
│                                                                                                  │
│ /home/tianbai_cui_siriusxm_com/disk2/miniconda3/envs/torch/lib/python3.10/site-packages/transfor │
│ mers/utils/hub.py:661 in _create_repo                                                            │
│                                                                                                  │
│    658 │   │   │   │   │   repo_id = repo_id.split("/")[-1]                                      │
│    659 │   │   │   │   repo_id = f"{organization}/{repo_id}"                                     │
│    660 │   │                                                                                     │
│ ❱  661 │   │   url = create_repo(repo_id=repo_id, token=use_auth_token, private=private, exist_  │
│    662 │   │                                                                                     │
│    663 │   │   # If the namespace is not there, add it or `upload_file` will complain            │
│    664 │   │   if "/" not in repo_id and url != f"{HUGGINGFACE_CO_RESOLVE_ENDPOINT}/{repo_id}":  │
│                                                                                                  │
│ in _inner_fn:120                                                                                 │
│                                                                                                  │
│   117 │   │   if check_use_auth_token:                                                           │
│   118 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   119 │   │                                                                                      │
│ ❱ 120 │   │   return fn(*args, **kwargs)                                                         │
│   121 │                                                                                          │
│   122 │   return _inner_fn  # type: ignore                                                       │
│   123                                                                                            │
│                                                                                                  │
│ in create_repo:2181                                        